### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. THis application will translate text from ENglish into another language. This is a relatively simple LLM application- it;s just a single  LLm call plus some prompting. Still, this is a  great wat to get started with Langchain - a lot of features can be built with just some prompting and an LLm call!

High level overview of :
- Using language models
- Using PrommptTempplates and OutputParsers
- Using LangChain Expression Language(LCEL) to chain components together
- Debugging and tracing your application using LangSmith
- Deploying application with LangServe


In [11]:
### OpenAI API Key and open Source models -- Llama3, Gemma2, mistral -- Groq
import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key = os.getenv("OPENAI_API_KEY")


#### Groq
##### What is the LPU Inference Engine ?
An LPU Interface Engine, with LPU Standing for Language Processing Unit, is a hardware and software platform that delivers exceptional compute speed, quality and energy efficiency. This new type of end-to-end processing unit system provides the fastest inference for computationally intensive application with sequential componenets, such as AI lnaguage application like Large Lnaguage Models(LLMs).
##### Why is it so much faster than GPUs for LLMs and GenAI?
The LPU is designed to overcome the two LLM bottlenecks: Compute density and memory bandwidth. An LPU has greater compute capacity than a GPU and CPU in regards to LLMs. This reduces the amount of time per word calculated, allowing sequence to text to be generated much faster.

In [12]:
groq_api_key  = os.getenv("GROQ_KEY")
groq_api_key

'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'

In [16]:
!pip install langchain_core


In [13]:
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI

In [15]:
model = ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001EAD615AC90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001EAD615A0C0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [18]:
from langchain_core.messages import HumanMessage,SystemMessage
messages = [
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Helloe How are you?")
]
result = model.invoke(messages)

In [19]:
result

AIMessage(content="The correct translation is:\n\n**Bonjour, comment allez-vous ?** \n\nHere's a breakdown:\n\n* **Bonjour:** Hello\n* **comment:** how\n* **allez-vous:**  are you (formal, polite) \n\n\nLet me know if you have any other phrases you'd like translated!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 22, 'total_tokens': 92, 'completion_time': 0.127272727, 'prompt_time': 0.000149169, 'queue_time': 0.01296388, 'total_time': 0.127421896}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-e3d0101d-e502-4fcd-a07e-25590c4b862f-0', usage_metadata={'input_tokens': 22, 'output_tokens': 70, 'total_tokens': 92})

In [21]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parser.invoke(result)

"The correct translation is:\n\n**Bonjour, comment allez-vous ?** \n\nHere's a breakdown:\n\n* **Bonjour:** Hello\n* **comment:** how\n* **allez-vous:**  are you (formal, polite) \n\n\nLet me know if you have any other phrases you'd like translated!\n"

In [22]:
### USing LCEL - chain the components
chain = model|parser
chain.invoke(messages)

"The correct translation is: \n\n**Bonjour, comment ça va ?** \n\nHere's a breakdown:\n\n* **Bonjour** - Hello\n* **Comment ça va ?** - How are you? (informal)\n\n\nLet me know if you have any other phrases you'd like translated!\n"

In [23]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template = "Translate the following into {language}:"

prompt = ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)


In [25]:
result = prompt.invoke({"language":"French","text":"Hello"})

In [26]:
result.to_messages()

[SystemMessage(content='Translate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [30]:
## Chaiing together components with LCEL
cahin = prompt | model |parser
cahin.invoke({"language":"French","text":"Hello"})

'Hello in French is **Bonjour**. \n'

### LangServe
LangServe helps developers deploy LangChain runnables and chains as a REST API.This library is integrated with FastAPI and uses pydantic for data validation.In addition, it provides a client that can be used to call into runnables deployed on a server. 

In [31]:
!pip install fastapi

In [32]:
!pip install uvicorn

In [33]:
!pip install langserve

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ----------------- ---------------------- 0.5/1.2 MB 5.6 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 4.9 MB/s eta 0:00:00


In [38]:
!pip install langchain_groq